## This notebook creates intensity-matrix heatmaps

In [1]:
from pathlib import Path
from definitions import ROOT_DIR
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch

- Inputs

In [2]:
p_root_dir = Path(ROOT_DIR).parents[0]
p_data = p_root_dir / "5_data_analysis"
p_out = p_root_dir / "6_plots/q2_plots/heatmaps"

# Compounds name and ID information
p_compounds = p_data / "compounds_ids.csv"

# Metrics and Catboost predictions for all ions in their target wells
p_predictions = p_data / "all_predictions_12-Jul-2021.csv"

# Dataset info (lab, matrix, polarity, m/z range, ids, etc.)
p_datasets = p_data / "datasets.csv"

# Classification
p_chem_class = p_data / "custom_classification.csv"
p_alt_class = p_data / "alternative_classification.csv"
p_pathways = p_data / "pathways.csv"

- Load and merge predictions with dataset metadata and compound classification

In [3]:
predictions = pd.read_csv(p_predictions, index_col=0)
predictions.neutral_loss.fillna('', inplace=True)

compounds = pd.read_csv(p_compounds, index_col='internal_id')
chem_class = pd.read_csv(p_chem_class, index_col='internal_id')
alt_class = pd.read_csv(p_alt_class, index_col='internal_id')
pathways = pd.read_csv(p_pathways, index_col='internal_id')

# Get a subset of most relevant information from Datasets file
datasets = pd.read_csv(p_datasets)
datasets_info = datasets.groupby('Clone ID').first()[['Polarity', 'Matrix short', 'Matrix long']] # 'Participant lab', 'Technology'

# Merge with predictions and classification
df = pd.merge(predictions, datasets_info, left_on='dataset_id', right_on='Clone ID', how='left')
df = pd.merge(df, chem_class, how='left', on='name_short')
df = pd.merge(df, alt_class, how='left', on='name_short')
df = pd.merge(df, pathways, how='left', on='name_short')

- Prepare data in two ways:
    1. Sum intensities of all ion forms (adducts and neutral losses)
    2. Sum intensities only of common adducts, but do not include any neutral losses

In [4]:
# Make columns with intensities for only ion images classified as 'good'
# For the plots below we are going to use background-corrected intensity, ie. (mean_on_spot - mean_bgr)
df['val_twostate'] = (df.pred_twostate == 1) * df.spot_intensity_bgr_corrected
df['val_threestate'] = (df.pred_threestate == 2) * df.spot_intensity_bgr_corrected

In [5]:
# Group full data by compound
grouped_data = df.groupby(['Matrix short', 'Polarity', 'name_short'])

agg_data = grouped_data.agg({
            'coarse_class' : 'first',
            'fine_class' : 'first',
            'morgan_class':'first', 
            'hmdb_func_group_class':'first', 
            'property_class':'first',
            'coarse_path' : 'first',
            'fine_path' : 'first',
            'val_twostate' : lambda x: np.log10(sum(x)+1),
            'val_threestate' : lambda x: np.log10(sum(x)+1),
})
agg_data.reset_index(inplace=True)

In [6]:
# Group filtered data
nl_list = ['']
df_slim = df[df.neutral_loss.isin(nl_list)]

grouped_data_slim = df_slim.groupby(['Matrix short', 'Polarity', 'name_short'])

agg_data_slim = grouped_data_slim.agg({
            'coarse_class' : 'first',
            'fine_class' : 'first',
            'morgan_class':'first', 
            'hmdb_func_group_class':'first', 
            'property_class':'first',
            'coarse_path' : 'first',
            'fine_path' : 'first',
            'val_twostate' : lambda x: np.log10(sum(x)+1),
            'val_threestate' : lambda x: np.log10(sum(x)+1),
})
agg_data_slim.reset_index(inplace=True)

- Safety check if any compounds are missing

In [7]:
# all_compounds = compounds[compounds.hmdb_primary != 'custom'].name_short
# missing = all_compounds[~all_compounds.isin(agg_data_slim.name_short)]
# if len(missing > 0):
#     print(f"{missing} compounds are missing results")

### Part 1: Hierarchical culstering in Seaborn

**Option 1**

Make separate plots for:
1. Full or filtered data    
2. Ionisation mode
3. Catboost prediction confidence level

In [ ]:
# Choose full or filtered data
for data, data_label in zip([agg_data, agg_data_slim], ['with n.l.', 'no n.l']):
    
    # Define a colour palette for chemical classes
    lut = dict(zip(data.coarse_class.unique(), sns.hls_palette(data.coarse_class.nunique(), h=.5)))
    my_classes = pd.Series(data.coarse_class.values, index=data.name_short).to_dict()
       
    # choose ionisation mode
    for mode in ['pos', 'neg']:
        data_filtered = data[data['Polarity'] == mode]
               
        # Choose catboost classification confidence level
        for catboost in ['twostate', 'threestate']:
            
            # Assemble data
            result = data_filtered.pivot(index='name_short',
                              columns=['Matrix short'],
                              values=f"val_{catboost}")
            result = result.reindex(data_filtered.name_short.drop_duplicates())
            result = result.fillna(0)
            result.to_csv(p_out / "hierarchical clustering modes separate" / f"data_for_seriation_{data_label}_{mode}_{catboost}.csv")
            
            # Make a colour bar corresponding to compound classes
            row_colors = result.index.map(my_classes).map(lut)
                        
            # Make a plot
            g = sns.clustermap(result, metric="euclidean", 
                               yticklabels=False, 
                               row_colors=row_colors,
                               row_cluster=True, 
                               cmap="Blues",
                               vmax=5.5) # set max value of the colour bar
                     
            # Beautify the plot
            ax = g.ax_heatmap # define axes
            ax.set(xlabel=None, ylabel=None) # hide axes labels
            g.ax_row_dendrogram.set_visible(False) # hiderow dendrogram
            
                # set title and give it space above the plot
            g.fig.subplots_adjust(top=0.9, right=0.95)
            g.fig.suptitle(f"log10 intensities {data_label} {mode} {catboost}") # set title
            
                # adjust colour bar size and location            
            g.ax_cbar.set_position((0.96, 0.075, 0.03, 0.65)) #x, y, w, l
            
                # construct legend
            handles = [Patch(facecolor=lut[name]) for name in lut]
            plt.legend(handles, lut, title='Chemical class',
                        bbox_to_anchor=(0.17, 0.5), bbox_transform=plt.gcf().transFigure, loc='upper right')

            # Save the lot
            g.savefig(p_out / "hierarchical clustering modes separate" / f"log10 intensities {data_label} {mode} {catboost}.png")
            plt.close()            

**Option 2**

Make separate plots for:
1. Full or filtered data
2. Catboost prediction confidence level

In [ ]:
# Choose full or filtered data
for data, data_label in zip([agg_data, agg_data_slim], ['with n.l.', 'no n.l']):
    
    # Define a colour palette for chemical classes
    lut = dict(zip(data.coarse_class.unique(), sns.hls_palette(data.coarse_class.nunique(), h=.5)))
    lut_mode = dict(zip(['pos', 'neg'], sns.hls_palette(2, h=.5)))
    my_classes = pd.Series(data.coarse_class.values, index=data.name_short).to_dict()
               
    # Choose catboost classification confidence level
    for catboost in ['twostate', 'threestate']:
            
        # Assemble data
        result = data.pivot(index='name_short',
                              columns=['Matrix short', 'Polarity'],
                              values=f"val_{catboost}")
        result = result.reindex(data.name_short.drop_duplicates())
        result = result.fillna(0)
        result.to_csv(p_out / "hierarchical clustering modes together" / f"data_for_seriation_{data_label}_{catboost}.csv")
            
        # Make a colour bar corresponding to compound classes
        row_colors = result.index.map(my_classes).map(lut)
        # Make a colour bar corresponding to ionisation mode
        col_colors = pd.Series([x[1] for x in result.columns], index=result.columns).map(lut_mode)
                        
        # Make a plot
        g = sns.clustermap(result, metric="euclidean", 
                               yticklabels=False, 
                               row_colors=row_colors,
                               col_colors=col_colors,
                               row_cluster=True, 
                               cmap="Blues",
                               vmax=5.5) # set max value of the colour bar
                     
        # Beautify the plot
        ax = g.ax_heatmap # define axes
        ax.set(xlabel=None, ylabel=None) # hide axes labels
        g.ax_row_dendrogram.set_visible(False) # hiderow dendrogram
            
            # set title and give it space above the plot
        g.fig.subplots_adjust(top=0.9, right=0.95)
        g.fig.suptitle(f"log10 intensities {data_label} {catboost}") # set title
            
            # adjust colour bar size and location            
        g.ax_cbar.set_position((0.96, 0.075, 0.03, 0.65)) #x, y, w, l
            
            # construct legend
        handles = [Patch(facecolor=lut[name]) for name in lut]
        plt.legend(handles, lut, title='Chemical class',
                        bbox_to_anchor=(0.17, 0.5), bbox_transform=plt.gcf().transFigure, loc='upper right')
        # Save the plot
        g.savefig(p_out / "hierarchical clustering modes together" / f"log10 intensities {data_label} {catboost}.png")
        plt.close()            

### Part 2: Seriation in R

- Describe the process

### Part 3: Class-specific plots

**Option 1**

Make separate plots for:
1. Full or filtered data
2. Chemical class
3. Ionisation mode
4. Catboost prediction confidence level

In [13]:
# Choose full or filtered data
for data, data_label in zip([agg_data, agg_data_slim], ['with n.l.', 'no n.l']):
    
    # sort rows
    data.sort_values(['coarse_class', 'fine_class'], inplace=True)

    for clas in data.coarse_class.unique():

        data_filtered = data[data.coarse_class == clas]
        row_order = data_filtered.name_short.drop_duplicates()

        # Define colour look-up table for fine classes
        fine_class = data_filtered.fine_class
        lut = dict(zip(fine_class.unique(), sns.hls_palette(fine_class.nunique(), h=.5)))
        my_classes = pd.Series(data_filtered.fine_class.values, index=data_filtered.name_short).to_dict()

        for mode in ['pos', 'neg']:

            data_filtered2 = data_filtered[data_filtered['Polarity'] == mode]

            for catboost in ['twostate', 'threestate']:
                result = data_filtered2.pivot(index='name_short', columns='Matrix short', values=f"val_{catboost}")
                # Pivot can create pairs of compound-matrix with no data (no ion image), set these to 0
                result = result.fillna(0)
                # maintain sorted row order
                result = result.reindex(row_order)
                # Prepare vector for side colour bar
                row_colors = result.index.map(my_classes).map(lut)

                # Plot the plot
                g = sns.clustermap(result, metric="euclidean", 
                                   yticklabels=True, 
                                   row_colors=row_colors, 
                                   row_cluster=False,
                                   col_cluster=False,
                                   cmap="Blues",
                                   vmax=5.5)
                
                # Beautify the plot
                ax = g.ax_heatmap # define axes
                ax.set(xlabel=None, ylabel=None) # hide axes labels
                g.ax_row_dendrogram.set_visible(False) # hiderow dendrogram

                    # set title and give it space above the plot
                g.fig.subplots_adjust(top=0.9, right=0.95)
                g.fig.suptitle(f"{clas} log10 intensities {mode} {data_label} {catboost}") # set title

                    # adjust colour bar size and location            
                g.ax_cbar.set_position((0.14, 0.075, 0.03, 0.65)) #x, y, w, l

                    # construct legend
                handles = [Patch(facecolor=lut[name]) for name in lut]
                plt.legend(handles, lut, title='Chemical class',
                                bbox_to_anchor=(0.1, 0.5), bbox_transform=plt.gcf().transFigure, loc='upper right')
                # Save the plot
                g.savefig(p_out / "heatmaps no cluster" / f"Heatmap class {clas} {mode} {catboost} {data_label}.png")
                plt.close()